# Visualize with Makie

## Basic Makie

In [1]:
using DataFrames
import CairoMakie as MK
using AlgebraOfGraphics

df = DataFrame(id = 1:100, x = rand(100), y = rand(100))

Row,id,x,y
,Int64,Float64,Float64
1,1,0.195684,0.26312
2,2,0.561613,0.561183
3,3,0.194505,0.0909367
4,4,0.699895,0.890355
5,5,0.460965,0.986068
6,6,0.113299,0.542652
7,7,0.681189,0.165532
8,8,0.281367,0.503991
9,9,0.758798,0.356881


In [2]:
fig = MK.lines(df.x, df.y)
MK.scatter!(df.x, df.y, color = :red)
MK.save("makie.pdf", fig) # src

CairoMakie.Screen{PDF}


In [3]:
fig = MK.Figure(backgroundcolor = :tomato)
MK.Axis(fig[1,1], title = "First figure")
MK.scatter!(df.x, df.y, color = :red)
MK.Axis(fig[1,2], title = "Second figure")
MK.lines!(df.x, df.y, color = :blue)
MK.save("makie.pdf", fig) # src

CairoMakie.Screen{PDF}


In [4]:
fig = MK.scatter(df.x, df.y, color = :red, label = "group 1")
MK.scatter!(rand(100), rand(100), color = :blue, label = "group 2")
MK.axislegend()
MK.save("makie.pdf", fig) # src

CairoMakie.Screen{PDF}


In [5]:
X = rand(100, 80)

fig, ax, hm = MK.heatmap(X, colormap = MK.Reverse(:RdBu))
MK.Colorbar(fig[1,2], hm)
MK.save("makie.pdf", fig) # src

CairoMakie.Screen{PDF}


## Algebra of Graphics

In [6]:
df.group = rand(string.(1:4), 100)

100-element Vector{String}:
 "4"
 "4"
 "3"
 "3"
 "4"
 "2"
 "1"
 "2"
 "4"
 "1"
 ⋮
 "2"
 "3"
 "3"
 "4"
 "4"
 "2"
 "1"
 "1"
 "1"

In [7]:
g = data(df) * mapping(:x, :y) * visual(MK.Scatter)
fig = draw(g)
MK.save("makie.pdf", fig) # src

CairoMakie.Screen{PDF}


In [8]:
g = data(df) * mapping(:x, :y) * visual(MK.Lines, color = :red)
fig = draw(g)
MK.save("makie.pdf", fig) # src

CairoMakie.Screen{PDF}


In [9]:
g = data(df) * mapping(:x, :y, color = :group) *
    (visual(MK.Scatter) + visual(MK.Lines))
fig = draw(g)
MK.save("makie.pdf", fig) # src

CairoMakie.Screen{PDF}


In [10]:
g = data(df) * mapping(:x, :y, layout = :group, color = :group) *
    (visual(MK.Scatter) + visual(MK.Lines))
fig = draw(g)
MK.save("makie.pdf", fig) # src

CairoMakie.Screen{PDF}


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*